In [11]:
import pandas as pd
import numpy as np

In [12]:
import os

In [13]:
%pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Meyer\Documents\Dom_Detecter_Code\Dom_Detecter\dom_detecter_env\Scripts\python.exe -m pip install --upgrade pip' command.


In [15]:
df = pd.read_excel(r"C:\Users\Meyer\Documents\Dom_Detecter\flow_data\sampled_df_unsw_23_02.xlsx_train_with_features.xlsx")

In [16]:
df_merged = df

In [ ]:
# Set Pandas to display all columns and rows
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [18]:
# Count the number of rows
num_rows = df_merged.shape[0]

# Print the number of rows
print(f"The DataFrame has {num_rows} rows.")

The DataFrame has 312000 rows.


In [19]:
# Count the number of packets for each target
target_counts = df_merged['target'].value_counts()

# Print the counts
print("Number of packets for each target:")
print(target_counts)

Number of packets for each target:
target
amazon_echo           26000
belkin_motion         26000
belkin_switch         26000
netatmo_cam           26000
netatmo_weather       26000
pixstar_photoframe    26000
samsung_cam           26000
smart_things          26000
tp-link_cam           26000
triby_speaker         26000
unknown               26000
withings_sleep        26000
Name: count, dtype: int64


In [20]:
flow_counts = df_merged.groupby('target')['Flow'].nunique()

In [21]:
print(flow_counts)

target
amazon_echo           5313
belkin_motion         5895
belkin_switch         5087
netatmo_cam           2263
netatmo_weather       1203
pixstar_photoframe    5749
samsung_cam           4299
smart_things           700
tp-link_cam            932
triby_speaker          149
unknown               4170
withings_sleep        1364
Name: Flow, dtype: int64


In [ ]:
# Drop duplicate rows based on the 'Flow' column, keeping only the first occurrence
df_unique = df_merged.drop_duplicates(subset=['Flow'], keep='first')

In [ ]:
# Count the number of packets for each target
target_counts = df_unique['target'].value_counts()

# Print the counts
print("Number of packets for each target:")
print(target_counts)

Number of packets for each target:
amazon_echo           9480
unknown               8032
samsung_cam           4977
belkin_motion         3952
pixstar_photoframe    2950
belkin_switch         2029
netatmo_cam           1516
smart_things          1402
tp-link_cam            827
netatmo_weather        490
tp-link_plug           376
triby_speaker           75
hp_printer              33
nest_smoke              21
withings_scale           3
blipcare_bp_meter        2
Name: target, dtype: int64


In [ ]:
# Count the number of packets for each target
target_counts = filtered_df['target'].value_counts()

# Print the counts
print("Number of packets for each target:")
print(target_counts)

Number of packets for each target:
amazon_echo           9480
unknown               8032
samsung_cam           4977
belkin_motion         3952
pixstar_photoframe    2950
belkin_switch         2029
netatmo_cam           1516
smart_things          1402
tp-link_cam            827
netatmo_weather        490
triby_speaker           75
Name: target, dtype: int64


In [ ]:
# Empty DataFrame to hold the balanced data
balanced_df = pd.DataFrame()

for target, group in filtered_df.groupby('target'):
    if len(group) > 1100:
        balanced_df = pd.concat([balanced_df, group.sample(n=1100, random_state=42)], axis=0)
    else:
        balanced_df = pd.concat([balanced_df, group], axis=0)

# Now balanced_df holds the balanced data

In [ ]:
# Count the number of packets for each target
target_counts = balanced_df['target'].value_counts()

# Print the counts
print("Number of packets for each target:")
print(target_counts)

Number of packets for each target:
amazon_echo           1100
belkin_motion         1100
belkin_switch         1100
netatmo_cam           1100
pixstar_photoframe    1100
samsung_cam           1100
smart_things          1100
unknown               1100
tp-link_cam            827
netatmo_weather        490
triby_speaker           75
Name: target, dtype: int64


In [ ]:
def split_group(group):
    if len(group) > 1000:
        # Randomly sample 1000 rows for the test set
        test = group.sample(n=1000, random_state=1)
        # Use drop to remove the rows that were selected for the test set from the group to form the holdout set
        holdout = group.drop(test.index)
        return test, holdout
    else:
        # For groups with fewer than 1000 rows, return the entire group as the test set and an empty DataFrame as the holdout set
        return group, pd.DataFrame()

# Initialize empty DataFrames for the test and holdout sets
df_test = pd.DataFrame()
df_holdout = pd.DataFrame()

# Split each group and append to df_test and df_holdout
for name, group in balanced_df.groupby('target'):
    test, holdout = split_group(group)
    df_test = df_test.append(test)
    df_holdout = df_holdout.append(holdout)

# Reset the index of df_test and df_holdout
df_test.reset_index(drop=True, inplace=True)
df_holdout.reset_index(drop=True, inplace=True)

<ipython-input-25-cf0aaee088fa>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test = df_test.append(test)
<ipython-input-25-cf0aaee088fa>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_holdout = df_holdout.append(holdout)
<ipython-input-25-cf0aaee088fa>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test = df_test.append(test)
<ipython-input-25-cf0aaee088fa>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_holdout = df_holdout.append(holdout)
<ipython-input-25-cf0aaee088fa>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_

In [ ]:
# Count the number of packets for each target
target_counts = df_test['target'].value_counts()

# Print the counts
print("Number of packets for each target:")
print(target_counts)

Number of packets for each target:
amazon_echo           1000
belkin_motion         1000
belkin_switch         1000
netatmo_cam           1000
pixstar_photoframe    1000
samsung_cam           1000
smart_things          1000
unknown               1000
tp-link_cam            827
netatmo_weather        490
triby_speaker           75
Name: target, dtype: int64


In [ ]:
df_test.to_excel('/gdrive/MyDrive/IomT/unsw_data/flow/def_flow_data/df_test_sampled_df_flow_23_28.xlsx', index=False)

In [ ]:
df_holdout.to_excel('/gdrive/MyDrive/IomT/unsw_data/flow/def_flow_data/df_holdout_23_28.xlsx', index=False)

In [ ]:
df_holdout.tail()

,idx,protocol,src_ip,src_port,dst_ip,dst_port,packet_length,tcp_flags,window_size,ttl,frame_len,dsfield_dscp,app_layer_protocol,dsfield_ecn,flags_df,flags_mf,ip_hdr_len,ip_id,ip_len,udp_src_port,udp_dst_port,udp_length,udp_checksum_status,packet_eth_src,flow_label,traffic_class,next_header,payload_len,packet.sniff_timestamp,Flow,Packet_Count,Start_Time,End_Time,Duration,Flow_Volume,Flow_Rate,Inter_Arrival_Time,Total_Fwd_Packets,Total_Bwd_Packets,Fwd_Packet_Length_Total,Bwd_Packet_Length_Total,target
795,634943,UDP,fe80::d9f9:ebd9:7242:4961,64640.0,fe80::16cc:20ff:fe51:33ea,53.0,NaN,NaN,NaN,64.0,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,43.0,NaN,NaN,NaN,NaN,NaN,0x000000,0x00000000,17.0,43.0,1.474606e+09,fe80::16cc:20ff:fe51:33ea:53-fe80::d9f9:ebd9:7...,4,1.474606e+09,1.474606e+09,0.003544,419,118232.869752,0.0,1,1,97,322,unknown
796,559744,UDP,fe80::d9f9:ebd9:7242:4961,58767.0,fe80::16cc:20ff:fe51:33ea,53.0,NaN,NaN,NaN,64.0,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,46.0,NaN,NaN,NaN,NaN,NaN,0x000000,0x00000000,17.0,46.0,1.474601e+09,fe80::16cc:20ff:fe51:33ea:53-fe80::d9f9:ebd9:7...,4,1.474601e+09,1.474601e+09,0.003792,554,146104.402414,0.0,1,1,100,454,unknown
797,563037,UDP,192.168.1.248,49826.0,192.168.1.1,53.0,97.0,NaN,NaN,64.0,97.0,0.0,DNS,0.0,1.0,0.0,20.0,0xc8c2,83.0,49826.0,53.0,63.0,2.0,b4:ce:f6:a7:a3:c2,NaN,NaN,NaN,NaN,1.474601e+09,192.168.1.1:53-192.168.1.248:49826-UDP,4,1.474601e+09,1.474601e+09,0.005899,411,69673.387115,0.0,1,1,97,314,unknown
798,882300,UDP,fe80::a21:efff:fe3b:fce3,58946.0,fe80::16cc:20ff:fe51:33ea,53.0,NaN,NaN,NaN,64.0,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,40.0,NaN,NaN,NaN,NaN,NaN,0x000000,0x00000000,17.0,40.0,1.474629e+09,fe80::16cc:20ff:fe51:33ea:53-fe80::a21:efff:fe...,4,1.474629e+09,1.474629e+09,0.003332,390,117050.344186,0.0,1,1,94,296,unknown
799,146137,UDP,192.168.1.166,38456.0,192.168.1.1,53.0,79.0,NaN,NaN,64.0,79.0,0.0,DNS,0.0,1.0,0.0,20.0,0x2f81,65.0,38456.0,53.0,45.0,2.0,00:24:e4:11:18:a8,NaN,NaN,NaN,NaN,1.474749e+09,192.168.1.166:38456-192.168.1.1:53-UDP,4,1.474749e+09,1.474749e+09,0.000319,174,545447.605381,0.0,1,1,79,95,unknown
